# Clean Up the Dataset and Categorize the Columns

In [1]:

import numpy as np
import pandas as pd
import utils

CONFIG_FILE = "01-02-clean-dataset_config.yml"


In [2]:
# Load Notebook Config
config = utils.load_config(CONFIG_FILE)
config

{'general': {'load_from_scratch': False,
  'save_raw_dataframe': False,
  'save_transformed_dataframe': False,
  'remove_bad_values': True},
 'columns': {'categorical': ['neighbourhood_group',
   'neighbourhood',
   'room_type'],
  'continuous': ['minimum_nights',
   'number_of_reviews',
   'reviews_per_month',
   'calculated_host_listings_count',
   'latitude',
   'longitude'],
  'date': ['last_review'],
  'text': ['name', 'host_name'],
  'excluded': ['price', 'id']},
 'bounding_box': {'max_long': -73.70018092,
  'max_lat': 40.91617849,
  'min_long': -74.25909008,
  'min_lat': 40.47739894},
 'newark_bounding_box': {'max_long': -74.11278706,
  'max_lat': 40.67325015,
  'min_long': -74.25132408,
  'min_lat': 40.78813864},
 'geo_columns': ['latitude', 'longitude'],
 'file_names': {'input_csv': '../data/AB_NYC_2019.csv',
  'pickle_input_dataframe': '../data/AB_NYC_2019_input_13_sep_2023.pkl',
  'pickle_output_dataframe': '../data/AB_NYC_2019_output_13_sep_2023.pkl'}}

1. In a fresh Python Jupyter notebook, ingest the DataFrame that you saved as a pickle file in the previous milestone into a DataFrame.

In [3]:
raw_df = pd.read_pickle(config["file_names"]["pickle_input_dataframe"])

In [4]:
raw_df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


## Clean Categorical Columns

In [5]:
# Create a copy of the dataframe to save the cleaned version
clean_df = raw_df.copy()

In [6]:
columns_categorical = config["columns"]["categorical"]

raw_df[columns_categorical].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   neighbourhood_group  48895 non-null  object
 1   neighbourhood        48895 non-null  object
 2   room_type            48895 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


There are no missing values for categorical columns, so we'll simply convert the columns into categorical

In [7]:
for column in columns_categorical:
    clean_df[column] = pd.Categorical(clean_df[column])

## Clean continuous columns

In [8]:
columns_continuous = config["columns"]["continuous"]

raw_df[columns_continuous].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   minimum_nights                  48895 non-null  int64  
 1   number_of_reviews               48895 non-null  int64  
 2   reviews_per_month               38843 non-null  float64
 3   calculated_host_listings_count  48895 non-null  int64  
 4   latitude                        48895 non-null  float64
 5   longitude                       48895 non-null  float64
dtypes: float64(3), int64(3)
memory usage: 2.2 MB


Note: reviews_per_month column has a number of missing values. We could replace these with 0, which would indicate no reviews.

In [9]:
clean_df["reviews_per_month"] = clean_df["reviews_per_month"].fillna(0)

Ensure `minimum_nights`, `number_of_reviews`, `reviews_per_month` and `calculated_host_listing_count` columns have positive values. If they are negative, set them to 0.

In [10]:
for column in ["minimum_nights", "number_of_reviews", "reviews_per_month", "calculated_host_listings_count"]:
    invalid_rows = clean_df[column] < 0
    clean_df[column] = np.where(invalid_rows, 0, clean_df[column])
    print(f"{column} column had {sum(invalid_rows)} negative values, which has been set to 0.")

minimum_nights column had 0 negative values, which has been set to 0.
number_of_reviews column had 0 negative values, which has been set to 0.
reviews_per_month column had 0 negative values, which has been set to 0.
calculated_host_listings_count column had 0 negative values, which has been set to 0.


Ensure `latitude` and `longitudes` are within the NYC bounding box

In [22]:

num_outside_box = sum(
    (raw_df["latitude"] < config["bounding_box"]["min_lat"]) |
    (raw_df["latitude"] > config["bounding_box"]["max_lat"]) |
    (raw_df["longitude"] < config["bounding_box"]["min_long"]) |
    (raw_df["longitude"] > config["bounding_box"]["max_long"])
)

print(f"There are {num_outside_box} rows with latitude and longitude outside NYC bounding box.")

There are 0 rows with latitude and longitude outside NYC bounding box.


## Clean date column
Convert `last_review` column to date time. Note: we'll keep missing values as is, because they are literally missing and cannot be substituted if there were no reviews.

In [11]:
clean_df["last_review"] = pd.to_datetime(clean_df["last_review"])

## Clean text columns

Replace missing values with a blank string.

In [12]:
for column in config["columns"]["text"]:
    num_missing = sum(raw_df[column].isna())
    clean_df[column] = clean_df[column].fillna("")
    print(f"{column} column had {num_missing} missing values, they were replaced with ''")

name column had 16 missing values, they were replaced with ''
host_name column had 21 missing values, they were replaced with ''


## Clean excluded columns

In [13]:
raw_df[config["columns"]["excluded"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   price   48895 non-null  int64
 1   id      48895 non-null  int64
dtypes: int64(2)
memory usage: 764.1 KB


Ensure `price` column is not less than 0.

In [14]:
sum(raw_df["price"] < 0)

0

## Save cleaned dataset

In [15]:
save_path = config["file_names"]["pickle_output_dataframe"]
clean_df.to_pickle(save_path)

print(f"Cleanded Dataframe saved at: {save_path}")

Cleanded Dataframe saved at: ../data/AB_NYC_2019_output_13_sep_2023.pkl
